In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, SimpleRNN

# Ch15_RNN과 CNN을 사용해 시퀀스 처리하기

## 순환 신경망(Recurrent Neural Network, RNN) 요약
- `Hands-on Machine Learning`의 내용 중에서 중요하다고 생각되는 내용이나 새롭게 알게 된
내용을 간단하게 요약해서 정리하고자 한다.
- 시간이 충분하지는 않아서 그림과 같은 자세한 설명은 생략하려고 한다.

## 15.1 순환 뉴런과 순환 층
- RNN은 매 타임 스텝 t마다 모든 뉴런은 입력 벡터 $x_{(t)}$와 타임 스텝의 출력 벡터 $y_{(t-1)}$을 받는다.
- 각 순환 뉴런은 입력 벡터 $x_{(t)}$를 위한 가중치 벡터 $w_x$와 출력 벡터 $y_{(t-1)}$를 위한 가중치 벡터 $w_y$를 가진다.
- 이를 순환 층 전체로 생각하면 각각 가중치 행렬 $W_x$, $W_y$가 된다.
- 순환 층 전체의 출력 벡터는 아래와 같은 식으로 표현된다.
- 여기서 $\mathbf{b}$는 편향이고 $\phi$는 활성 함수이다.
> $\mathbf{y}_{(t)}=\phi(\mathbf{W}_{x}^{T}\mathbf{x}_{(t)}+\mathbf{W}_{x}^{T}\mathbf{y}_{(t-1)}+\mathbf{b})$
- 식을 보면, $\mathbf{y}_{(t)}$는 이전 상태의 출력인 $\mathbf{y}_{(t-1)}$도 입력 받기 때문에, 결국 가장 첫 번째 입력인 $\mathbf{x}_{0}$에 대한 값까지도 가지고 있게 된다.
- 첫번째 타임 스텝 t=0에서는 이전 출력이 없으므로 모두 0이라고 가정한다.

### 15.1.1 메모리 셀
- 타입 스텝 t에서 순환 뉴런의 출력은 이전 타임 스텝의 모든 입력에 대한 값이므로 일종의 메모리 형태이다.
- 타임 스텝에 걸쳐서 어떤 상태를 보존하는 신경망의 구성 요소를 메모리 셀(memory cell, 또는 셀)이라고 한다.
- 타임 스텝 t에서의 셀의 상태 $\mathbf{h}_{(t)}$(hidden cell)는 그 타임 스텝의 입력과 이전 타임 스텝의 상태에 대한 함수 $\mathbf{h}_{(t)}=f(\mathbf{h}_{(t-1)}, \mathbf{x}_{(t)})$로 나타낼 수 있다.

### 15.1.2 입력과 출력 시퀀스
- Sequence-to-sequence network: 입력 시퀀스를 받아 출력 시퀀스를 만드는 RNN으로, 시계열 데이터를 예측하는데 유용하다.
- Sequence-to-vector network: 입력 시퀀스를 받아 마지막 출력 벡터만 만드는 RNN이다.
- Vector-to-sequence network: 각 타임 스텝에서 하나의 입력 벡터를 반복해서 입력하고 하나의 시퀀스를 출력하는 RNN으로,
    예를 들어 이미지를 입력하여 이미지에 대한 캡션을 출력하는 RNN이 있다.
- encoder-decoder: 인코더라 부르는 Sequence-to-vector network 뒤에 디코더라 부르는 Vector-to-sequence network를 연결하는 구조로, 문장 번역에 자주 사용된다.
    - 한 언어의 문장을 네트워크에 입력하면, 인코더는 이 문장을 하나의 벡터로 변환하고, 디코더는 이 벡터를 받아 다른 언어의 문장으로 디코딩한다.
    - 문장의 마지막 단어가 번역 시 첫번째 단어에 영향을 주는 경우가 많기 때문에, 인코더-디코더와 같은 이중 RNN은 하나의 Sequence-to-vector network보다 훨씬 더 잘 작동한다고 한다.

## 15.2 RNN 훈련하기
- RNN을 학습시킬 때에는 BPTT(Backpropagation Through Time)을 통해 학습하므로, 그레디언트가 마지막 출력뿐만 아니라 손실 함수를 사용한 모든 출력에 대해 역전파된다.
- 또한 각 타임 스텝마다 같은 매개변수 $\mathbf{W}$와 $\mathbf{b}$가 사용되기 때문에 순전파에서 모두 동일한 가중치가 적용되어 계산이 진행된 후 역전파가 진행되면 모든 타임 스텝에 걸쳐 합산된다.

## 15.3 시계열 예측하기
- 시계열(Time series) 데이터: 타임 스텝마다 하나 이상의 값을 가지는 시퀀스 데이터.
    - 단변량 시계열(Univariate time series): 타임 스텝마다 하나의 값을 가지는 데이터.
    - 다변량 시계열(Multivariate time series): 타임 스텝마다 여러 값을 가지는 데이터.
- 시계열을 다룰 때 입력 특성은 일반적으로 [배치 크기, 타임 스텝 수, 차원 수] 크기의 3D 배열로 나타낸다고 한다.
- 따라서 단변량 시계열의 경우 차원 수는 1이 되고, 다변량 시계열의 경우 차원 수가 1 이상이 된다.

In [2]:
# 시계열 생성해주는 함수이다.
# batch_size만큼 n_steps 길이의 여러 시계열을 만든다.
# [배치 크기, 타임 스텝 수, 1] 크기의 넘파이 배열을 리턴하므로 이 시계열 데이터는 단변량 데이터이다.
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # 사인 곡선 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 사인 곡선 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

In [55]:
# 위의 함수를 이용하여 훈련셋, 검증셋, 테스트셋을 생성한다.
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### 15.3.1 기준 성능
- RNN과 비교할 비교군으로 사용할 모델들을 생성한다.
1. 순진한 예측(Naive forecasting): 각 시계열의 마지막 값을 그대로 예측
    - 이렇게 하는 것도 높은 확률이 될 수 있다.

In [23]:
y_pred = X_valid[:, -1]
mse = np.mean(tf.keras.losses.mean_squared_error(y_valid, y_pred))
print(f"mse: {mse}")

mse: 0.02011820673942566


2. MLP(완전 연결층) 사용
    - 검증셋에 대한 mse를 확인해보면, 순진한 예측보다는 좋은 결과를 출력하는 것을 알 수 있다.

In [14]:
model = Sequential([
    Flatten(input_shape=[50, 1]),
    Dense(1)
])

model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())

history = model.fit(X_train, y_train, epochs=20, verbose=False)
mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"MSE of MLP: {mse}")

2000/1 - 0s - loss: 0.0042
MSE of MLP: 0.004281704217195511


### 15.3.2 간단한 RNN 구현하기
- `SimpleRNN()`은 하나의 뉴런으로 이루어진 하나의 층을 가지는 RNN 구조이다.
- `SimpleRNN()`은 기본적으로 `tanh`를 활성 함수로 사용한다.
- RNN은 어떤 길이의 타임 스텝도 처리할 수 있기 때문에 입력 시퀀스의 길이를 지정할 필요가 없다.
- __return_sequences=True__로 옵션을 줄 경우 모든 타임 스텝마다 출력을 반환해준다고 한다.

In [15]:
model = Sequential([
    SimpleRNN(units=1, input_shape=[None, 1])
])

model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())
model.fit(X_train, y_train, epochs=20, verbose=0)
rnn_mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"MSE of SimpleRNN: {rnn_mse}")

2000/1 - 0s - loss: 0.0126
MSE of SimpleRNN: 0.011292242005467416


- 위의 결과를 보면, `SimpleRNN`이 `MLP`보다 좋지 못한 성능을 보여준다는 것을 알 수 있다.
- 이는 `SimpleRNN`의 구조가 너무 간단하여 데이터에 과소적합되기 때문이다.
- 생각해보면, `MLP`의 경우 모든 특징 벡터에 편향을 더한 수인 총 51개의 파라미터를 가지지만, `SimpleRNN`의 경우 매 타임 스텝마다의 입력과 곱하는 가중치 1개, 이전 상태의 값과 곱하는 가중치 1개에 편향을 더해 총 3개의 파라미터만을 가진다.

### 15.3.3 심층 RNN
- RNN은 기존의 인공신경망과 같이 셀을 여러 층으로 쌓는 것이 일반적이고, 이를 심층 RNN(deep RNN)이라고 한다.
- `return_sequences=True`로 지정하지 않을 경우 (모든 타임 스텝에 대한 출력을 담은) 3D 배열이 아닌 
(마지막 타임 스텝의 출력만 담은) 2D 배열이 리턴되므로 다음 `SimpleRNN`의 입력 형태와 맞지 않게 된다.
- 따라서 모든 `SimpleRNN` 층에서 `return_sequences=True`로 지정해주어야 한다(마지막 출력만 관심 대상일 경우에는 
아래와 같이 마지막 층에서는 설정하지 않으면 된다.).

In [37]:
model = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(1)
])

model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=1)
drnn_mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"MSE of deep RNN: {drnn_mse}")

Train on 7000 samples
Epoch 1/20
7000/7000 [==============================] - 5s 770us/sample - loss: 0.0306
Epoch 2/20
7000/7000 [==============================] - 3s 411us/sample - loss: 0.0068
Epoch 3/20
7000/7000 [==============================] - 3s 421us/sample - loss: 0.0046
Epoch 4/20
7000/7000 [==============================] - 3s 407us/sample - loss: 0.0039
Epoch 5/20
7000/7000 [==============================] - 3s 411us/sample - loss: 0.0035
Epoch 6/20
7000/7000 [==============================] - 3s 406us/sample - loss: 0.0032
Epoch 7/20
7000/7000 [==============================] - 3s 409us/sample - loss: 0.0031
Epoch 8/20
7000/7000 [==============================] - 3s 410us/sample - loss: 0.0030
Epoch 9/20
7000/7000 [==============================] - 3s 406us/sample - loss: 0.0029
Epoch 10/20
7000/7000 [==============================] - 3s 410us/sample - loss: 0.0029
Epoch 11/20
7000/7000 [==============================] - 3s 412us/sample - loss: 0.0028
Epoch 12/20
7000/70

- 출력 결과를 보면, `deep RNN`이 `MLP`보다 성능이 더 좋은 것을 확인할 수 있다.
- 그러나 위의 모델의 경우 마지막 층까지 RNN을 사용하였는데, 마지막 층의 은닉 상태는 크게 필요하지 않는다고 한다.
- 이는 RNN이 한 타임 스텝에서 다음 타임 스텝으로 필요한 정보를 나를 때 마지막 층이 아닌 다른 층의 은닉 상태를 주로 사용할
것이기 때문이라고 한다.
- 아마 마지막 층에 해당하는 은닉 상태의 경우 단지 출력값을 내는데에만 사용되기 때문이라는 말인 것 같다.
- 또한 `SimpleRNN`의 경우 `tanh`를 활성 함수로 사용하기 때문에 예측값이 -1 ~ 1 사이에 놓이게 되므로 이 또한 유용하지 않다,
- 이러한 이유로 출력층의 경우 `Dence` 층으로 사용하는 경우가 많다고 한다.
- 이에 대한 코드는 아래와 같다.

In [38]:
model = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20),
    Dense(1)
])

model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=128, verbose=2)
denseRNN_mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"MSE of deep RNN: {denseRNN_mse}")

Train on 7000 samples
Epoch 1/20
7000/7000 - 4s - loss: 0.0756
Epoch 2/20
7000/7000 - 2s - loss: 0.0143
Epoch 3/20
7000/7000 - 2s - loss: 0.0075
Epoch 4/20
7000/7000 - 2s - loss: 0.0052
Epoch 5/20
7000/7000 - 2s - loss: 0.0043
Epoch 6/20
7000/7000 - 2s - loss: 0.0037
Epoch 7/20
7000/7000 - 2s - loss: 0.0034
Epoch 8/20
7000/7000 - 2s - loss: 0.0033
Epoch 9/20
7000/7000 - 2s - loss: 0.0031
Epoch 10/20
7000/7000 - 2s - loss: 0.0030
Epoch 11/20
7000/7000 - 2s - loss: 0.0030
Epoch 12/20
7000/7000 - 2s - loss: 0.0030
Epoch 13/20
7000/7000 - 2s - loss: 0.0029
Epoch 14/20
7000/7000 - 2s - loss: 0.0029
Epoch 15/20
7000/7000 - 2s - loss: 0.0028
Epoch 16/20
7000/7000 - 2s - loss: 0.0028
Epoch 17/20
7000/7000 - 2s - loss: 0.0028
Epoch 18/20
7000/7000 - 2s - loss: 0.0028
Epoch 19/20
7000/7000 - 2s - loss: 0.0028
Epoch 20/20
7000/7000 - 2s - loss: 0.0028
2000/1 - 1s - loss: 0.0030
MSE of deep RNN: 0.0027124519646167756


- 위와 같이 마지막 층을 `Dense` 층으로 바꿀 경우 훈련 시간은 줄어듦과 동시에 비슷한 정확도를 낼 수 있다.

In [ ]:
### 15.3.4 여러 타임 스텝 앞을 예측하기
- RNN을 훈련하여 다음 값 10개를 한 번에 예측할 수도 있다.
- 이 경우 Sequence-to-vector network를 사용하면 된다.
- 또한 데이터셋도 타깃값의 갯수가 10개가 되도록 바꾸어주어야 한다.

In [56]:
series = generate_time_series(10000, n_steps + 10)
X_train, y_train = series[:7000, :n_steps], series[:7000, -10:, 0]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -10:, 0]
X_test, y_test = series[9000:, :n_steps], series[9000:, -10:, 0]

In [59]:
model = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20),
    Dense(10)
])

model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, verbose=1)
vec_mse = model.evaluate(X_valid, y_valid, verbose=2)
print(f"mse of 10 outputs: {vec_mse}")

Train on 7000 samples
Epoch 1/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0530
Epoch 2/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0217
Epoch 3/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0167
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0137
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0120
Epoch 6/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0113
Epoch 7/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0106
Epoch 8/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0101
Epoch 9/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0100
Epoch 10/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0098
Epoch 11/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0094
Epoch 12/20
7000/7000 [==================

- 출력 결과를 보면, 다음 10개 타임 스텝에 대한 MSE는 약 0.008로, 추가하지는 않았지만 책에서 기술한 MLP의 MSE 0.0188보다
좋은 성능을 보임을 알 수 있다.
- 위의 sequence-to-vector network는 마지막 타임 스텝에서만 다음 값 10개의 벡터를 예측값으로 출력한다.
- 이 대신 모든 타임 스텝에서 다음 값 10개를 예측하도록 모델을 sequence-to-sequence network로 만들 수도 있다.
- 즉, 타임 스텝 0에서 모델이 타임 스텝 1에서 10까지의 예측값을 출력하고, 타임 스텝 1에서는 타임 스텝 2에서 11까지의
예측값을 출력하는 식으로 계속한다는 의미이다.
- sequence-to-sequence network로 모델을 생성할 경우 마지막 타임 스텝에서의 출력뿐만 아니라 모든 타임 스텝에서
RNN 출력에 대한 항이 loss에 포함되므로 더 많은 loss의 그레디언트가 모델로 흐르게 되고, 또한 각 타임 스텝의 출력에서도
그레디언트가 흐를 수 있다.
- 이를 통해 훈련을 더 안정적으로 만들고 훈련 속도도 높아진다고 한다.

- sequence-to-sequence network를 적용하기 위해서는 타깃의 형태도 이에 맞게 아래와 같이 수정해주어야 한다.

In [60]:
Y = np.empty((10000, n_steps, 10)) # 각 타깃은 10D 벡터의 시퀀스이다.
for step_ahead in range(1, 10 + 1):
    Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

- sequence-to-sequence network 모델을 사용하려면 모든 RNN 층을 return_sequences=True로 설정해주어야 한다(매 타임 스텝마다 출력을 내야 하므로).
- 그다음 모든 타임 스텝에서 출력을 Dense 층에 적용해야 하는데, 이 때 케라스의 TimeDistributed 층을 사용한다.
- TimeDistributed 층은 다른 층(예제에서는 Dense 층)을 감싸서 입력 시퀀스의 모든 타임 스텝에 해당 층을 적용한다.
- 각 타임 스텝을 별개의 샘플처럼 다루도록 Dense 층에 입력하기 전에 입력의 크기를 바꿔주고(Dense 층의 경우 1D 입력을 받으므로) Dense 층을 적용한다.
- Dense 층을 적용한 후에는 출력 크기를 다시 시퀀스로 되돌린다.
- 예시를 통한 설명은 p.612를 확인하는 것이 좋다.
- 훈련하는 동안에는 모든 출력이 필요하지만, 예측과 평가에는 마지막 타임 스텝의 출력만 필요하다.
- 따라서 평가를 위해서 마지막 타임 스텝의 출력에 대한 MSE만을 계산하는 함수를 작성하였다.

In [65]:
model = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[last_time_step_mse])
model.fit(X_train, Y_train, epochs=20, verbose=1)
mse, last_mse = model.evaluate(X_valid, Y_valid, verbose=2)
print(f"last_mse: {last_mse}")

Train on 7000 samples
Epoch 1/20
7000/7000 [==============================] - 9s 1ms/sample - loss: 0.0483 - last_time_step_mse: 0.0370
Epoch 2/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0347 - last_time_step_mse: 0.0224
Epoch 3/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0317 - last_time_step_mse: 0.0205
Epoch 4/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0284 - last_time_step_mse: 0.0162
Epoch 5/20
7000/7000 [==============================] - 8s 1ms/sample - loss: 0.0258 - last_time_step_mse: 0.0134
Epoch 6/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0241 - last_time_step_mse: 0.0119
Epoch 7/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0225 - last_time_step_mse: 0.0102
Epoch 8/20
7000/7000 [==============================] - 7s 1ms/sample - loss: 0.0215 - last_time_step_mse: 0.0092
Epoch 9/20
7000/7000 [==============================] - 7s 1ms/sam

In [ ]:
- 검증셋을 통해 MSE를 확인한 결과 약 0.006으로 기존의 마지막 타임 스텝에서만 출력을 내던 모델보다 약 25% 향상된 성능을
보인다.

In [69]:
Y_pred = model.predict(X_test)

In [70]:
keras.losses.mean_squared_error(Y_test, Y_pred)

<tf.Tensor: id=144593, shape=(1000, 50), dtype=float32, numpy=
array([[0.23754124, 0.10417585, 0.05781896, ..., 0.0028493 , 0.00125886,
        0.00167957],
       [0.13193132, 0.0323742 , 0.03554861, ..., 0.00253879, 0.00355834,
        0.00347603],
       [0.20612144, 0.13787611, 0.16158494, ..., 0.00888486, 0.00454423,
        0.00263105],
       ...,
       [0.04095913, 0.06381775, 0.06147507, ..., 0.0082857 , 0.01838318,
        0.01420282],
       [0.04653487, 0.03727365, 0.04552943, ..., 0.0009662 , 0.00106046,
        0.00285792],
       [0.15506986, 0.05730684, 0.03645291, ..., 0.00747423, 0.01185518,
        0.01875053]], dtype=float32)>